In [25]:
import sys
sys.path.append('/Users/gomez/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages')
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import requests
from config import g_key

In [26]:
city_data_df = pd.read_csv("challenge_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rainfall in past hour (inches),Snowfall in past hour (inches)
0,0,Barrow,US,71.29,-156.79,32.00,96,90,8.05,fog,0.0,0.0
1,1,Talnakh,RU,69.49,88.40,62.60,48,81,11.18,broken clouds,0.0,0.0
2,2,Cape Town,ZA,-33.93,18.42,64.99,59,4,10.29,clear sky,0.0,0.0
3,3,Ushuaia,AR,-54.80,-68.30,33.80,86,90,4.70,moderate rain,0.0,0.0
4,4,Rikitea,PF,-23.12,-134.97,69.35,61,70,13.56,broken clouds,0.0,0.0


In [27]:
gmaps.configure(api_key=g_key)

In [28]:
min_temperature = float(input("What is the minimum temperature that you would prefer for your vacation? "))

What is the minimum temperature that you would prefer for your vacation? 75


In [29]:
max_temperature = float(input("What is the maximum temperature that you would prefer for your vacation? "))

What is the maximum temperature that you would prefer for your vacation? 98


In [30]:
rain_factor = input("Do you want it to be raining? (Yes/No) ")

Do you want it to be raining? (Yes/No) no


In [31]:
snow_factor = input("Do you want it to be snowing? (Yes/No) ")

Do you want it to be snowing? (Yes/No) no


In [32]:
if rain_factor == "no" and snow_factor == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temperature) & \
                                       (city_data_df["Max Temp"] >= min_temperature) & \
                                       (city_data_df["Rainfall in past hour (inches)"] == 0) & \
                                        (city_data_df["Snowfall in past hour (inches)"] == 0)]
elif rain_factor == "no" and snow_factor == "yes":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temperature) & \
                                       (city_data_df["Max Temp"] >= min_temperature) & \
                                       (city_data_df["Rainfall in past hour (inches)"] == 0) & \
                                        (city_data_df["Snowfall in past hour (inches)"] >= 0.0)]
elif rain_factor == "yes" and snow_factor == "no":
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temperature) & \
                                       (city_data_df["Max Temp"] >= min_temperature) & \
                                       (city_data_df["Rainfall in past hour (inches)"] >= 0.0) & \
                                        (city_data_df["Snowfall in past hour (inches)"] == 0.0)]
else:
    filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temperature) & \
                                       (city_data_df["Max Temp"] >= min_temperature) & \
                                       (city_data_df["Rainfall in past hour (inches)"] >= 0.0) & \
                                        (city_data_df["Snowfall in past hour (inches)"] >= 0.0)]

In [33]:
filtered_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rainfall in past hour (inches),Snowfall in past hour (inches)
8,8,Nikolskoye,RU,59.70,30.79,77.00,33,0,4.47,clear sky,0.0,0.0
10,10,Muros,ES,42.78,-9.06,75.00,68,1,10.00,clear sky,0.0,0.0
18,18,Butaritari,KI,3.07,172.79,82.80,75,57,13.31,broken clouds,0.0,0.0
20,20,Tiznit,MA,29.58,-9.50,78.91,53,56,8.19,broken clouds,0.0,0.0
25,25,Kashi,CN,39.45,75.98,87.80,14,40,17.90,scattered clouds,0.0,0.0


In [34]:
len(filtered_cities_df)

216

In [35]:
filtered_cities_df.count()

City_ID                           216
City                              216
Country                           213
Lat                               216
Lng                               216
Max Temp                          216
Humidity                          216
Cloudiness                        216
Wind Speed                        216
Description                       216
Rainfall in past hour (inches)    216
Snowfall in past hour (inches)    216
dtype: int64

In [36]:
hotel_df = filtered_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Nikolskoye,RU,77.00,59.70,30.79,
10,Muros,ES,75.00,42.78,-9.06,
18,Butaritari,KI,82.80,3.07,172.79,
20,Tiznit,MA,78.91,29.58,-9.50,
25,Kashi,CN,87.80,39.45,75.98,


In [37]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [38]:
for index, row in hotel_df.iterrows():
    # Get latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [39]:
hotel_df.count()

City          216
Country       213
Max Temp      216
Lat           216
Lng           216
Hotel Name    216
dtype: int64

In [40]:
len(hotel_df)

216

In [41]:
hotel_df = hotel_df.dropna()

In [42]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Nikolskoye,RU,77.00,59.70,30.79,Tourist House - Sablino
10,Muros,ES,75.00,42.78,-9.06,Apartment Av. Castelao
18,Butaritari,KI,82.80,3.07,172.79,Isles Sunset Lodge
20,Tiznit,MA,78.91,29.58,-9.50,
25,Kashi,CN,87.80,39.45,75.98,帕米尔青年旅舍


In [43]:
output_data_file = "challenge_data/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [44]:
vacation_df = pd.read_csv("challenge_data/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,8,Nikolskoye,RU,77.00,59.70,30.79,Tourist House - Sablino
1,10,Muros,ES,75.00,42.78,-9.06,Apartment Av. Castelao
2,18,Butaritari,KI,82.80,3.07,172.79,Isles Sunset Lodge
3,20,Tiznit,MA,78.91,29.58,-9.50,NaN
4,25,Kashi,CN,87.80,39.45,75.98,帕米尔青年旅舍


In [49]:
vacation_df = vacation_df.dropna()

In [46]:
vacation_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Hotel Name
0,8,Nikolskoye,RU,77.00,59.70,30.79,Tourist House - Sablino
1,10,Muros,ES,75.00,42.78,-9.06,Apartment Av. Castelao
2,18,Butaritari,KI,82.80,3.07,172.79,Isles Sunset Lodge
4,25,Kashi,CN,87.80,39.45,75.98,帕米尔青年旅舍
5,26,Cabo Rojo,PR,84.99,18.09,-67.15,Hotel Perichi's


In [47]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
output_data_file = "challenge_data/WeatherPy_vacation.csv"
hotel_df.to_csv(output_data_file, index_label = "City_ID")